In [1]:
# 首先导入所需第三方库
# from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import UnstructuredFileLoader
# from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
import os

In [2]:
# 获取文件路径函数
def get_files(dir_path):
    # args：dir_path，目标文件夹路径
    file_list = []
    for filepath, dirnames, filenames in os.walk(dir_path):
        # os.walk 函数将递归遍历指定文件夹
        for filename in filenames:
            # 通过后缀名判断文件类型是否满足要求
            if filename.endswith(".md"):
                # 如果满足要求，将其绝对路径加入到结果列表
                file_list.append(os.path.join(filepath, filename))
            elif filename.endswith(".txt"):
                file_list.append(os.path.join(filepath, filename))
    return file_list

In [3]:
# 加载文件函数
def get_text(dir_path):
    # args：dir_path，目标文件夹路径
    # 首先调用上文定义的函数得到目标文件路径列表
    file_lst = get_files(dir_path)
    # docs 存放加载之后的纯文本对象
    docs = []
    # 遍历所有目标文件
    for one_file in tqdm(file_lst):
        file_type = one_file.split('.')[-1]
        if file_type == 'md':
            loader = UnstructuredMarkdownLoader(one_file)
        elif file_type == 'txt':
            loader = UnstructuredFileLoader(one_file)
        else:
            # 如果是不符合条件的文件，直接跳过
            continue
        docs.extend(loader.load())
    return docs

In [4]:
# 目标文件夹
tar_dir = [
    "./data/InternLM",
    "./data/InternLM-XComposer",
    "./data/lagent",
    "./data/lmdeploy",
    "./data/opencompass",
    "./data/xtuner"
]

In [7]:
# 加载目标文件
docs = []
for dir_path in tar_dir:
    docs.extend(get_text(dir_path))
docs[:5]

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 38/38 [00:02<00:00, 15.06it/s]


[Document(page_content='', metadata={'source': './data/InternLM\\CHANGE_LOG.md'}),
 Document(page_content='InternLM\n\nInternLM\n\nHOT\n\nはじめに\n\nInternLM は、70 億のパラメータを持つベースモデルと、実用的なシナリオに合わせたチャットモデルをオープンソース化しています。このモデルには以下の特徴があります:\n\n何兆もの高品質なトークンをトレーニングに活用し、強力な知識ベースを確立します。\n\n8k のコンテキストウィンドウ長をサポートし、より長い入力シーケンスと強力な推論機能を可能にする。\n\nユーザが独自のワークフローを柔軟に構築できるよう、汎用性の高いツールセットを提供します。\n\nさらに、大規模な依存関係を必要とせずにモデルの事前学習をサポートする軽量な学習フレームワークが提供されます。単一のコードベースで、数千の GPU を持つ大規模クラスタでの事前学習と、単一の GPU での微調整をサポートし、顕著な性能最適化を達成します。InternLM は、1024GPU でのトレーニングにおいて 90% 近いアクセラレーション効率を達成しています。\n\n新闻\n\nInternLM-7B-Chat v1.1 は、コード インタプリタと関数呼び出し機能を備えてリリースされました。 Lagent で試すことができます。\n\nInternLM-7B\n\nパフォーマンス評価\n\nオープンソースの評価ツール OpenCompass を用いて、InternLM の総合的な評価を行った。この評価では、分野別能力、言語能力、知識能力、推論能力、理解能力の 5 つの次元をカバーしました。以下は評価結果の一部であり、その他の評価結果については OpenCompass leaderboard をご覧ください。\n\nデータセット\\モデル InternLM-Chat-7B InternLM-7B LLaMA-7B Baichuan-7B ChatGLM2-6B Alpaca-7B Vicuna-7B C-Eval(Val) 53.2 53.4 24.2 42.7 50.9 28.9 31.2 MMLU 50.8 51.

In [8]:
# 对文本进行分块
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=150)
text_splitter

In [9]:
split_docs = text_splitter.split_documents(docs)
split_docs[:5]

[Document(page_content='InternLM\n\nInternLM\n\nHOT\n\nはじめに\n\nInternLM は、70 億のパラメータを持つベースモデルと、実用的なシナリオに合わせたチャットモデルをオープンソース化しています。このモデルには以下の特徴があります:\n\n何兆もの高品質なトークンをトレーニングに活用し、強力な知識ベースを確立します。\n\n8k のコンテキストウィンドウ長をサポートし、より長い入力シーケンスと強力な推論機能を可能にする。\n\nユーザが独自のワークフローを柔軟に構築できるよう、汎用性の高いツールセットを提供します。\n\nさらに、大規模な依存関係を必要とせずにモデルの事前学習をサポートする軽量な学習フレームワークが提供されます。単一のコードベースで、数千の GPU を持つ大規模クラスタでの事前学習と、単一の GPU での微調整をサポートし、顕著な性能最適化を達成します。InternLM は、1024GPU でのトレーニングにおいて 90% 近いアクセラレーション効率を達成しています。\n\n新闻', metadata={'source': './data/InternLM\\README-ja-JP.md'}),
 Document(page_content='新闻\n\nInternLM-7B-Chat v1.1 は、コード インタプリタと関数呼び出し機能を備えてリリースされました。 Lagent で試すことができます。\n\nInternLM-7B\n\nパフォーマンス評価\n\nオープンソースの評価ツール OpenCompass を用いて、InternLM の総合的な評価を行った。この評価では、分野別能力、言語能力、知識能力、推論能力、理解能力の 5 つの次元をカバーしました。以下は評価結果の一部であり、その他の評価結果については OpenCompass leaderboard をご覧ください。', metadata={'source': './data/InternLM\\README-ja-JP.md'}),
 Document(page_content='データセット\\モデル InternLM-Chat-7B InternLM-7B LLaMA-7B Baichuan-7B ChatGLM

In [10]:
# 加载开源词向量模型
embeddings = HuggingFaceEmbeddings(model_name="./sentence-transformer")

d:\miniconda3\envs\mm\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
# 构建向量数据库
# 定义持久化路径
persist_directory = './vector_db/chroma'
# 加载数据库
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

In [12]:
# 将加载的向量数据库持久化到磁盘上
vectordb.persist()